# Import statements

# Import dataset

## Load dataset

Verify if dataset has been loaded

## Split data and labels

Verify if data and labels have been splitted

## Split training and validation sets

Verify if training and validation sets are splitted

## Preprocess data

# Develop model

## *Iteration 1*

### Build model

### Visualize model

### Compile model

### Train model

### Plot results

# Test model